# [Soynlp] 학습 기반 토크나이저
- 품사 태깅, 단어 토큰화 등을 지원하는 단어 토크나이저
- 비지도 학습으로 단어 토큰화 -> 데이터에 자주 등장하는 단어들을 단어로 분석
- 내부적으로 단어 점수 표로 동작

In [2]:
from konlpy.tag import Okt

In [4]:
tokenizer = Okt()

print(tokenizer.morphs("에이비식스 이대휘 1월 최애돌 기부 요정 입니다."))

# 형태소 분석 시 매개변수 stem, norm 둘 다 default는 False 
print(tokenizer.morphs("에이비식스 이대휘 1월 최애돌 기부 요정 입니다.", stem = True)) # 어간
print(tokenizer.morphs("에이비식스 이대휘 1월 최애돌 기부 요정 입니다.", norm = True)) # 정규화 

['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정', '입니다', '.']
['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정', '이다', '.']
['에이', '비식스', '이대', '휘', '1월', '최애', '돌', '기부', '요정', '입니다', '.']


In [5]:
# [soynlp] 사용 -> 말뭉치 데이터셋 
from urllib.request import urlretrieve
import os

In [7]:
filename = "../data/text_data.txt"

In [8]:
# 학습 데이터 처리
from soynlp import DoublespaceLineCorpus # 한개로 통합된 문서 데이터를 분리하기 위함 
from soynlp.word import WordExtractor # 단어 추출 

In [9]:
# 훈련 데이터 문서 분리
corpus = DoublespaceLineCorpus(filename)
print(f"훈련 데이터 문서 : {len(corpus)}개")

훈련 데이터 문서 : 30091개


In [10]:
# SoyNLP 학습 진행
word_extractor = WordExtractor()

# 학습 진행하며 단어별 점수
word_extractor.train(corpus) # 단어마다 점수표를 만듦

# 단어별 점수표 추출
word_score_table = word_extractor.extract()  # 위에서 만든 점수표 추출
# print하면 컴퓨터 죽을 수도 있음 ㅠㅠ

training was done. used memory 1.083 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


In [11]:
# 단어별 점수표 확인

for idx, key in enumerate(word_score_table.keys()):
    print(f"{idx} -> {key}")
    if idx==30 : break

0 -> 맴
1 -> 윳
2 -> 며
3 -> 코
4 -> 펑
5 -> 킬
6 -> 맘
7 -> 온
8 -> 뒷
9 -> 뿐
10 -> 졸
11 -> 앗
12 -> 꺾
13 -> 아
14 -> 퇴
15 -> 폄
16 -> 성
17 -> 밧
18 -> 해
19 -> 씻
20 -> 낚
21 -> 탄
22 -> 솟
23 -> 불
24 -> 샐
25 -> 깨
26 -> 쳤
27 -> 랩
28 -> 샌
29 -> 됨
30 -> 핏


# 응집확률 : 내부 문자열이 얼마나 응집하여 자주 등장하는지를 판단하는 척도
- 원리 : 문자열을 문자 단위로 분리, 왼쪽부터 순서대로 문자를 추가 각 문자열이 주어졌을 때 그 다음 문자가 나올 확률을 계산 / 누적곱 한 값
- 값이 높을 수록 : 전체 코퍼스에서 이 문자열 시퀀스 하나의 단어로 등장할 가능성 높음

In [14]:
word_score_table["바"].cohesion_forward

0

In [12]:
word_score_table["바다"].cohesion_forward

0.06393648140409527

In [13]:
word_score_table["바다에"].cohesion_forward # 바다보다 바다에로 등장할 확률이 높다는 뜻

0.11518621707955429

In [15]:
word_score_table["바다에서"].cohesion_forward

0.15551296569078008

# soyNLP의 L tokenizer
- 띄어쓰기 단위로 나눈 어절 토큰 : L토큰 + R토큰
- 예 : 공원에 -> 공원+에 / 공부하는 -> 공부 + 하는
- 분리 기준 : 점수가 가장 높은 L 토큰을 찾아내는 원리

In [23]:
from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()} # 딕셔너리 conprehension

# 그냥 쪼개지 말고 점수가 높은 것 기준으로 L토큰을 찾아달라
l_tokenizer = LTokenizer(scores=scores)
l_tokenizer.tokenize("국제사회와 우리의 노력들로 범죄를 척결하자", flatten=False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

# 최대 점수 토크나이저
- 띄어쓰기가 되어있지 않은 문장에서 점수가 높은 글자 시퀀스를 순차적으로 찾아내는 토크나이저
- 띄어쓰기가 되어있지 않은 문장을 넣어서 점수를 통해 토큰화 된 결과

In [25]:
from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']

# soyNLP를 이용한 반복되는 문자 정제
- ㅋㅋ, ㅎㅎ 등의 이모티콘의 경우 불필요하게 연속되는 경우 많음
- ㅋㅋ, ㅋㅋㅋㅋㅋㅋ와 같은 경우를 모두 서로 다른 단어로 처리하는 것은 불필요
- 반복되는 것은 하나로 정규화

In [31]:
from soynlp.normalizer import *
import string
print(string.punctuation)

print(emoticon_normalize("악ㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠ", num_repeats=1))
print(emoticon_normalize("악ㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠ", num_repeats=2))
print(emoticon_normalize("악ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠㅠㅠ", num_repeats=2))
print(emoticon_normalize("악ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ", num_repeats=2))

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
ㅋㅋㅋ영화존잼쓰ㅠ
ㅋㅋㅋ영화존잼쓰ㅠㅠ
ㅋㅋ영화존잼쓰ㅠㅠ
ㅋㅋ영화존잼쓰ㅠㅠ


In [32]:
print(repeat_normalize("와하하하하하하하하하핫", num_repeats=2))
print(repeat_normalize("와하하하하하하핫", num_repeats=2))
print(repeat_normalize("와하하하하핫", num_repeats=2))

와하하핫
와하하핫
와하하핫


In [37]:
from ckonlpy.tag import Twitter

twitter = Twitter()
twitter.morphs("은경이는 사무실로 갔습니다.")

['은', '경이', '는', '사무실', '로', '갔습니다', '.']

In [39]:
twitter.pos("은경이는 사무실로 갔습니다.")

[('은', 'Noun'),
 ('경이', 'Noun'),
 ('는', 'Josa'),
 ('사무실', 'Noun'),
 ('로', 'Josa'),
 ('갔습니다', 'Verb'),
 ('.', 'Punctuation')]

In [40]:
# 형태소 분석기에 사전 추가
twitter.add_dictionary("은경이", "Noun")

In [41]:
twitter.pos("은경이는 사무실로 갔습니다.")

[('은경이', 'Noun'),
 ('는', 'Josa'),
 ('사무실', 'Noun'),
 ('로', 'Josa'),
 ('갔습니다', 'Verb'),
 ('.', 'Punctuation')]